# Analysis of the data obtained from OpenAgrar

In [8]:
import pandas as pd
import json

## Load the full Openagrar dataset (including datasets and articles)

In [21]:
openagrar = pd.read_csv("/home/abdelmalak/Documents/FairAgro UC/repo/pilot-uc-textmining-metadata/data/OpenAgrar/outputs/output.csv", sep="|")

In [4]:
with open("/home/abdelmalak/Documents/FairAgro UC/OpenAgrar_data/output_datasets/output.json", "r") as data:
    openagrar_json= json.load(data)

In [7]:
for element in openagrar_json:
    id = element["header"]["identifier"].split("_")[-1]
    #print(id)
    if id == "00100742":
        # Access abstract text
        description = element['metadata']['resource']['descriptions']['description']
        if isinstance(description, list):
            abstract_text = ""
            for desc in description:
                if desc['@descriptionType'] == 'Abstract':
                    abstract_text = desc["#text"]
                    break
            if abstract_text == "":
                abstract_text = "Not Found"
        else:
            if description['@descriptionType'] == 'Abstract':
                abstract_text = description["#text"]
            else:
                abstract_text = "Not Found"
        abstract_text = abstract_text.replace('\n',  ' ')
        abstract_text = '"'+abstract_text+'"'
        
        title = element["metadata"]["resource"]["titles"]["title"]
        if isinstance(title, str):
            title = title
        elif isinstance(title, list):
            if isinstance(title[0], str):
                title = title[0]
            else:
                title = title[0]['#text']
        else:
            title = title['#text']
        
        print(title)
        break

Fatty acid analyses reveal differences in feeding ecology of North Sea squids that overlap in time and space


### Narrow down to only articles and ti or jki publishers

In [22]:
filtered_df = openagrar[openagrar['institute'].str.contains('ti|jki', case=False, na=False)]
articles = filtered_df[openagrar['Type'].str.contains('article', case=False, na=False)]

/tmp/ipykernel_42650/4054845267.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  articles = filtered_df[openagrar['Type'].str.contains('article', case=False, na=False)]


### Create a filtering criteria based on sub-institutes

In [23]:
institutes = [
    "Crop and Soil Science",
    "Strategies and Technology Assessment",
    "Biological Control",
    "Plant Protection in Horticulture and Urban Green",
    "Breeding Research on Agricultural Crops",
    "Breeding Research on Horticultural Crops"
]
pattern = "|".join(institutes)


### Apply the filtering criterea

In [24]:
filtered_institutes = articles[articles["authors_names"].str.contains(pattern, na=False)]

In [25]:
filtered_institutes = filtered_institutes[filtered_institutes['publication_year']>=2018]
len(filtered_institutes)

473

### Save the articles in a csv file in order to save them as text files

In [26]:
filtered_file = "/home/abdelmalak/Documents/FairAgro UC/repo/pilot-uc-textmining-metadata/data/OpenAgrar/outputs/final_articles.csv"
filtered_institutes.to_csv(filtered_file, sep='|')

## Load the datasets in OpenAgrar

In [16]:
openagrar_datasets = pd.read_csv("/home/abdelmalak/Documents/FairAgro UC/repo/pilot-uc-textmining-metadata/data/OpenAgrar/outputs/output.csv", sep="|")

In [17]:
datasets = openagrar_datasets[openagrar_datasets['institute'].str.contains('ti|jki', case=False, na=False)]
datasets = datasets[openagrar['Type'].str.contains('ResearchData', case=False, na=False)]
len(datasets)

/tmp/ipykernel_42650/3850706504.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  datasets = datasets[openagrar['Type'].str.contains('ResearchData', case=False, na=False)]


356

In [18]:
#filtered_datasets = datasets[datasets["authors_names"].str.contains(pattern, na=False)]
filtered_datasets = datasets[datasets['publication_year']>=2018]
len(filtered_datasets)

318

In [19]:
filtered_file = "/home/abdelmalak/Documents/FairAgro UC/repo/pilot-uc-textmining-metadata/data/OpenAgrar/outputs/final_datasets.csv"
filtered_datasets.to_csv(filtered_file, sep='|')

In [10]:
openagrar.loc[openagrar['ID']==100742]

,ID,title,abstract_text,publisher,publication_year,institute,authors_names,subjects,language,Type
4689,100742,Fatty acid analyses reveal differences in feed...,"""Climate-induced changes in marine ecosystems ...",Elsevier BV OpenAgrar,2024,ti,"[{'creatorName': {'@nameType': 'Personal', '#t...","['fatty acid composition', 'squid', 'climate c...",en,ResearchData


In [ ]:
openagrar.to_excel("output.xlsx")

## Analysis of the full dataset

In [ ]:
import ast
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import json
from collections import Counter

In [ ]:
len(openagrar)

In [ ]:
openagrar.head()

In [20]:
filtered_datasets['abstract_text'].value_counts().reset_index().head(10)

,abstract_text,count
0,Not Found,18
1,"""Not Found""",12
2,"""Quecksilber gehört zu den giftigsten Elemente...",7
3,"""Blei ist ein giftiges Schwermetall und weltwe...",7
4,"""Cadmium ist ein giftiges Schwermetall und wel...",7
5,"""Polyaromatische Kohlenwasserstoffe (PAH) sind...",6
6,"""Polychlorierte Biphenyle (PCB) sind industrie...",3
7,"""Per- und Polyfluoralkylchemikalien (PFAS) sin...",3
8,"""With the ongoing cost decrease of genotyping ...",2
9,"""Range expansions can lead to increased contac...",2


In [ ]:
# define Seaborn color palette to use 
sns.barplot(x="language",
           y="count",
           data=openagrar['language'].value_counts().reset_index())
# displaying chart 
plt.show()

In [ ]:
# define Seaborn color palette to use 
sns.barplot(x="institute",
           y="count",
           data=openagrar['institute'].value_counts().reset_index())
# displaying chart 
plt.show()

In [ ]:
filtered_df['institute'].value_counts().reset_index().head(10)

In [ ]:
subjects = filtered_df['subjects']
all_subjects = []
counter = 0
for sub in subjects: 
    try:
        if isinstance(sub, str):
            data = ast.literal_eval(sub.lower())
            if not isinstance(data, dict):
                all_subjects.extend(data)
                for h in data: 
                    if 'soil' in str(h):
                        coounter += 1
                        break
    except:
        #print(sub)
        continue
print(counter)

In [ ]:
final_subjects = []
for i in all_subjects:
    if isinstance(i, dict):
        continue
    else:
        final_subjects.append(i)

In [ ]:
subjects_conter= Counter(final_subjects)

In [ ]:
isinstance(subjects_conter, dict)

In [ ]:
total_soil_mentions = 0
for i, val in zip(subjects_conter.keys(), subjects_conter.values()):
    if 'crop' in i:
        print(i, val)
        total_soil_mentions += val
print(f'Total soil mentions = {total_soil_mentions}')

In [ ]:
subjects_conter.most_common()

In [ ]:
# Filter out rows that do not contain 'en'
#filtered_df = openagrar[openagrar['language'].str.contains('en', case=False, na=False)]

In [ ]:
filtered_df = openagrar[openagrar['institute'].str.contains('ti|jki', case=False, na=False)]
articles = filtered_df[openagrar['Type'].str.contains('article', case=False, na=False)]

In [ ]:
len(articles)

In [ ]:
filtered_df['authors_names'][0]

Show the distributions of years and the distributions of institutes

In [ ]:
# define Seaborn color palette to use 
plt.figure(figsize=(12, 8))
ax = sns.barplot(x="institute",
           y="count",
           data=filtered_df['institute'].value_counts().reset_index())

# Annotate each bar with its value
for p in ax.patches:
    ax.annotate(
        f'{int(p.get_height())}',  # The value to display
        (p.get_x() + p.get_width() / 2, p.get_height()),  # Position of the text (x, y)
        ha='center', va='center',  # Horizontal and vertical alignment
        fontsize=12, color='black',  # Font size and color
        xytext=(0, 10), textcoords='offset points'  # Offset the text slightly above the bar
    )
# displaying chart 
plt.show()

In [ ]:
# Set the figure size to make the plot fill the notebook cell
plt.figure(figsize=(12, 8))

# Create the barplot
ax = sns.barplot(x="publication_year", y="count", data=filtered_df['publication_year'].value_counts().reset_index())

# Rotate x-axis labels by 90 degrees for better readability
plt.xticks(rotation=90)
# Annotate each bar with its value
for p in ax.patches:
    ax.annotate(
        f'{int(p.get_height())}',  # The value to display
        (p.get_x() + p.get_width() / 2, p.get_height()),  # Position of the text (x, y)
        ha='center', va='center',  # Horizontal and vertical alignment
        fontsize=12, color='black',  # Font size and color
        xytext=(0, 10), textcoords='offset points'  # Offset the text slightly above the bar
    )
# Display the chart
plt.show()

In [ ]:
# Set up the figure size for the plot
plt.figure(figsize=(10, 6))

# Plot the distribution of text lengths directly using apply() within sns.histplot
sns.histplot(filtered_df['abstract_text'].apply(lambda x: len(x.split(' '))), kde=True, color='blue', bins=10)

# Add labels and title
plt.title('Distribution of Text Lengths', fontsize=16)
plt.xlabel('Text Length (Number of Characters)', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

# Display the plot
plt.show()

In [ ]:
filtered_file = "/home/abdelmalak/Documents/FairAgro UC/repo/pilot-uc-textmining-metadata/data/OpenAgrar/outputs/final.csv"
filtered_df.to_csv(filtered_file, sep='|')

In [ ]:
filtered_df = filtered_df[filtered_df['publication_year']>=2018]

In [ ]:
len(filtered_df)